In [8]:
import numpy as np
import pandas as pd

In [9]:
df = pd.read_csv('article_titles_urls.csv', sep='\n', header = None)

In [10]:
titles = df[0].str.split('/,', expand=True)

In [11]:
titles.head()

,0,1
0,https://mashable.com/2013/01/07/amazon-instant...,Amazon's Streaming Video Library Now a Little ...
1,https://mashable.com/2013/01/07/ap-samsung-spo...,AP's Twitter to Begin Displaying Sponsored Twe...
2,https://mashable.com/2013/01/07/apple-40-billi...,"Apple's App Store Passes 40 Billion Downloads,"
3,https://mashable.com/2013/01/07/astronaut-notr...,"This Astronaut Is Rooting for Notre Dame Tonight,"
4,https://mashable.com/2013/01/07/att-u-verse-apps,New U-Verse Apps Simplify Sharing Photos and V...


In [12]:
titles['url']= titles[0].str.replace('https','http')

In [13]:
titles.drop(columns = 0, axis = 1, inplace =True)

In [14]:
titles = titles[['url',1]]

In [15]:
df_total = pd.read_csv('OnlineNewsPopularity.csv')

In [16]:
df_total = df_total[['url',' shares']]

In [17]:
df_total

,url,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,711
2,http://mashable.com/2013/01/07/apple-40-billio...,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,505
...,...,...
39639,http://mashable.com/2014/12/27/samsung-app-aut...,1800
39640,http://mashable.com/2014/12/27/seth-rogen-jame...,1900
39641,http://mashable.com/2014/12/27/son-pays-off-mo...,1900
39642,http://mashable.com/2014/12/27/ukraine-blasts/,1100


In [18]:
df_total['url'] = df_total['url'].map(lambda x: str(x)[:-1])

In [19]:
titles_numberofshares_df = pd.merge(df_total,titles, how='inner', on=['url'])

In [20]:
titles_numberofshares_df.drop(['url'], inplace=True, axis=1)

In [21]:
final_df=titles_numberofshares_df.drop(titles_numberofshares_df.index[0])

In [22]:
final_df

,shares,1
1,593,Amazon's Streaming Video Library Now a Little ...
2,711,AP's Twitter to Begin Displaying Sponsored Twe...
3,1500,"Apple's App Store Passes 40 Billion Downloads,"
4,1200,"This Astronaut Is Rooting for Notre Dame Tonight,"
5,505,New U-Verse Apps Simplify Sharing Photos and V...
...,...,...
39446,1800,Samsung app aims to improve autistic children'...
39447,1900,"Seth Rogen, James Franco are planning to live-..."
39448,1900,App developer says 'Merry Christmas' by paying...
39449,1100,Two dead after Ukraine rocked by series of bla...


In [23]:
final_df['popular'] = final_df[' shares'].apply(lambda x: 1 if (x >= 1400) else 0)

In [24]:
final_df

,shares,1,popular
1,593,Amazon's Streaming Video Library Now a Little ...,0
2,711,AP's Twitter to Begin Displaying Sponsored Twe...,0
3,1500,"Apple's App Store Passes 40 Billion Downloads,",1
4,1200,"This Astronaut Is Rooting for Notre Dame Tonight,",0
5,505,New U-Verse Apps Simplify Sharing Photos and V...,0
...,...,...,...
39446,1800,Samsung app aims to improve autistic children'...,1
39447,1900,"Seth Rogen, James Franco are planning to live-...",1
39448,1900,App developer says 'Merry Christmas' by paying...,1
39449,1100,Two dead after Ukraine rocked by series of bla...,0


In [26]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [31]:
X = final_df[1].values
y = final_df['popular'].values

In [32]:
list_of_length = [len(a.split()) for a in X]

In [33]:
b = np.array(list_of_length)
b.mean() + 2*b.std()

12.959579240278043

In [34]:
X.shape
y.shape

(39450,)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [256]:
vocab_size = 5000
max_length = 14
embedding_dim = 128
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [257]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [258]:
len(word_index)
len(y_train)

31560

In [259]:
X_train_article_titles_sequences = tokenizer.texts_to_sequences(X_train)

In [260]:
X_train_article_titles_padded = pad_sequences(X_train_article_titles_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [261]:
X_train_article_titles_padded.shape

(31560, 14)

In [262]:
X_test_article_titles_sequences = tokenizer.texts_to_sequences(X_test)

In [263]:
X_test_article_titles_padded = pad_sequences(X_test_article_titles_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [264]:
X_test_article_titles_padded.shape

(7890, 14)

In [265]:
# model = tf.keras.Sequential([
#     # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
#     tf.keras.layers.Embedding(vocab_size, embedding_dim),
# #     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#     # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#     # use ReLU in place of tanh function since they are very good alternatives of each other.
#     tf.keras.layers.Dense(embedding_dim, activation='relu'),
#     # Add a Dense layer with 6 units and softmax activation.
#     # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])
# model.summary()

In [266]:
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# num_epochs = 10
# history = model.fit(X_train_article_titles_padded, y_train, epochs=num_epochs, validation_data=(X_test_article_titles_padded, y_test), verbose=2)

In [282]:
model1 = tf.keras.Sequential()

In [283]:
model1.add(tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embedding_dim, input_length = max_length))

In [284]:
model1.add(tf.keras.layers.LSTM(units = embedding_dim))

In [285]:
model1.add(tf.keras.layers.Dropout(0.2))

In [286]:
model1.add(tf.keras.layers.Dense(1, activation = 'relu'))

In [287]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [288]:
model1.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 14, 128)           640000    
_________________________________________________________________
lstm_16 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 129       
Total params: 771,713
Trainable params: 771,713
Non-trainable params: 0
_________________________________________________________________


In [289]:
num_epochs = 15
history = model1.fit(X_train_article_titles_padded, y_train, epochs=num_epochs, validation_data=(X_test_article_titles_padded, y_test), verbose=2)

Train on 31560 samples, validate on 7890 samples
Epoch 1/15
31560/31560 - 21s - loss: 0.7019 - accuracy: 0.5108 - val_loss: 0.6927 - val_accuracy: 0.5385
Epoch 2/15
31560/31560 - 19s - loss: 0.6745 - accuracy: 0.5779 - val_loss: 0.7025 - val_accuracy: 0.5257
Epoch 3/15
31560/31560 - 19s - loss: 0.6516 - accuracy: 0.6202 - val_loss: 0.7373 - val_accuracy: 0.5181
Epoch 4/15
31560/31560 - 19s - loss: 0.6369 - accuracy: 0.6368 - val_loss: 0.8766 - val_accuracy: 0.5048
Epoch 5/15
31560/31560 - 19s - loss: 0.6108 - accuracy: 0.6616 - val_loss: 0.7169 - val_accuracy: 0.5122
Epoch 6/15
31560/31560 - 19s - loss: 0.5966 - accuracy: 0.6783 - val_loss: 0.7534 - val_accuracy: 0.4646
Epoch 7/15
31560/31560 - 19s - loss: 0.5820 - accuracy: 0.6903 - val_loss: 0.8056 - val_accuracy: 0.5077
Epoch 8/15
31560/31560 - 19s - loss: 0.5634 - accuracy: 0.7058 - val_loss: 1.3744 - val_accuracy: 0.5100
Epoch 9/15
31560/31560 - 19s - loss: 0.5197 - accuracy: 0.7380 - val_loss: 1.8304 - val_accuracy: 0.5090
Epoch 